### 使用PaddleDetection的 Faster RCNN 模型，对给出的脚本进行修改

In [ ]:
# 解压文件并移除多余的目录
! unzip /home/aistudio/data/data105746/train.zip -d /home/aistudio/data/steel
# !rm -r /home/aistudio/data/steel/__MACOSX
! unzip /home/aistudio/data/data105747/test.zip -d /home/aistudio/data/steel
# !rm -r /home/aistudio/data/steel/__MACOSX

In [ ]:
# 修改文件名字 JPEGImages  Annotations
!mv ./data/steel/train/ANNOTATIONS  ./data/steel/train/Annotations
!mv ./data/steel/train/IMAGES  ./data/steel/train/JPEGImages

使用paddleX拆分数据集

In [ ]:

!paddlex --split_dataset --format VOC --dataset_dir /home/aistudio/data/steel/train --val_value 0.001 --test_value 0.0

In [ ]:
# 进入PaddleDetection
%cd /home/aistudio/work/PaddleDetection
# 安装其它依赖
!pip install -r /home/aistudio/work/PaddleDetection/requirements.txt  
# 临时环境安装
!pip install pycocotools -i https://mirror.baidu.com/pypi/simple
!pip install lap -i https://mirror.baidu.com/pypi/simple

In [ ]:
%cd /home/aistudio/work/PaddleDetection/
#转换train
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir /home/aistudio/data/steel/train/ \
--voc_anno_list /home/aistudio/data/steel/train/train_list.txt \
--voc_label_list /home/aistudio/data/steel/train/labels.txt \
--voc_out_name /home/aistudio/data/steel/train/voc_train.json

#转换test
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir /home/aistudio/data/steel/train/ \
--voc_anno_list /home/aistudio/data/steel/train/val_list.txt \
--voc_label_list /home/aistudio/data/steel/train/labels.txt \
--voc_out_name /home/aistudio/data/steel/train/voc_val.json

!rm -r /home/aistudio/data/steel/train/Annotations/*
!mv /home/aistudio/data/steel/train/*.json /home/aistudio/data/steel/train/Annotations/

部分参数按照提交的两个.yml文件进行修改，

In [ ]:
# 训练
!python tools/train.py -c configs/faster_rcnn/faster_rcnn_swin_tiny_fpn_3x_coco.yml --use_vdl=true --vdl_log_dir=vdl_dir/scalar --eval

生成比赛数据，运行的最优模型已提交，需要修改一下模型路径

In [ ]:
# 推理图像和生成txt文件
!python tools/infer.py -c  configs/faster_rcnn/faster_rcnn_swin_tiny_fpn_3x_coco.yml \
-o weights=/home/aistudio/model/model \
--infer_dir=/home/aistudio/data/steel/test/IMAGES/ \
--output_dir=/home/aistudio/data/steel/infer_output\
--draw_threshold=0.005 --save_txt=True

In [ ]:
import csv
import os
headers = ['image_id','bbox','category_id','confidence']
classList = ['crazing','inclusion','pitted_surface','scratches','patches','rolled-in_scale']
rows = []

rootdir = '/home/aistudio/data/steel/infer_output'
list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
for i in range(0,len(list)):
       path = os.path.join(rootdir,list[i])
       if os.path.isfile(path) and path.endswith('txt'):
           txtFile = open(path)
           print(path)
           result = txtFile.readlines()
           for r in result:
               ls = r.split(' ')
               Cls = ls[0]
               sco = float(ls[1])
               xmin = float(ls[2])
               ymin = float(ls[3])
               w = float(ls[4])
               h = float(ls[5])
               xmax = xmin+w
               ymax = ymin+h
               clsID = classList.index(Cls)
               imgID = list[i][:-4]
               row = [imgID,[xmin,ymin,xmax,ymax],clsID,sco]
               rows.append(row)
with open('submission.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

In [ ]:
import pandas as pd
datafile = pd.read_csv('/home/aistudio/work/PaddleDetection/submission.csv')
# 按照列值排序
data = datafile.sort_values(by="image_id", ascending=True)
data.to_csv('submission_final.csv', mode='a+', index=False)